Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Create Image
In this notebook, we show the following steps for deploying a web service using AzureML:
- Create an image
- Test image locally

In [ ]:
from azure_utils.machine_learning.utils import load_configuration, get_workspace_from_config
from azure_utils.utilities import text_to_json
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model

AML will use the following information to create an image, provision a cluster and deploy a service. Replace the 
values in the following cell with your information.

In [ ]:
cfg = load_configuration("../workspace_conf.yml")

In [ ]:
image_name = cfg['image_name']

## Get workspace
Load existing workspace from the config file.

In [ ]:
ws = get_workspace_from_config()
print(ws.name, ws.resource_group, ws.location, sep="\n")

## Load model

In [ ]:
model_name = 'question_match_model'

model = Model(ws, name=model_name)
print(model.name, model.version)

## Create an image
We will now modify the `score.py` created in the previous notebook for the `init()` function to use the model we 
registered to the workspace earlier.

In [ ]:
%%writefile score.py

import sys
import pandas as pd
import json
from duplicate_model import DuplicateModel
import logging
import timeit as t
from azureml.core.model import Model
from azureml.contrib.services.aml_request import rawhttp

sys.path.append('./scripts/')


def init():
    logger = logging.getLogger("scoring_script")
    global model
    model_name = 'question_match_model'
    model_path = Model.get_model_path(model_name)
    questions_path = './data_folder/questions.tsv'
    start = t.default_timer()
    model = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)

@rawhttp
def run(request):
    """
    Function runs on each request
    """
    body = request.data
    if request.method == 'POST':
        logger = logging.getLogger("scoring_script")
        json_load_text = json.loads(body)
        text_to_score = json_load_text['input']
        start = t.default_timer()
        resp = model.score(text_to_score) 
        end = t.default_timer()
        logger.info("Prediction took {0} ms".format(round((end-start)*1000, 2)))
        return(json.dumps(resp))
    if request.method == 'GET':
        resp_body = {
            "azEnvironment": "Azure",
            "location": "westus2",
            "osType": "Ubuntu 16.04",
            "resourceGroupName": "",
            "resourceId": "",
            "sku": "",
            "subscriptionId": "",
            "uniqueId": "PythonMLRST",
            "vmSize": "",
            "zone": "",
            "isServer": False,
            "version": ""
        }
        return(resp_body)
    return AMLResponse("bad request", 500)

Let's specifiy the conda and pip dependencies for the image.

In [ ]:
conda_pack = ["scikit-learn==0.19.1", "pandas==0.23.3"]
requirements = ["lightgbm==2.1.2", "azureml-defaults==1.0.57", "azureml-contrib-services"]

In [ ]:
lgbmenv = CondaDependencies.create(conda_packages=conda_pack, pip_packages=requirements)

with open("lgbmenv.yml", "w") as f:
    f.write(lgbmenv.serialize_to_string())

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script="score.py",
    runtime="python",
    conda_file="lgbmenv.yml",
    description="Image with lightgbm model",
    tags={"area": "text", "type": "lightgbm"},
    dependencies=[
        "./data_folder/questions.tsv",
        "./duplicate_model.py",
        "./scripts/item_selector.py",
    ],
)

image = ContainerImage.create(
    name=image_name,
    # this is the model object
    models=[model],
    image_config=image_config,
    workspace=ws,
)

In [ ]:
%%time
image.wait_for_creation(show_output = True)

In [ ]:
print(image.name, image.version)

In [ ]:
image_version = str(image.version)

You can find the logs of image creation in the following location.

In [ ]:
image.image_build_log_uri

## Test image locally

Now, let's use one of the duplicate questions to test our image.

In [ ]:
dupes_test_path = './data_folder/dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0, 4]
text_to_score

In [ ]:
json_text = text_to_json(text_to_score)

In [ ]:
%%time
image.run(input_data=json_text)

## Conclusion

We have created a docker Image using AzureML and registred this image on Azure Container Registry (ACR). This docker 
image encapsulates a trained machine learning model and scoring scripts. In the next step, we can take this image 
and deploy it on the compute target of your choice: Azure Kubernetes Service (AKS) Cluster or Azure IoT Edge.